# Train Training Tweets

In [1]:
import json
import urllib
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from pandas.io.json import json_normalize
import datetime as dt

import csv
from nltk import word_tokenize, wordpunct_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.probability import FreqDist, DictionaryProbDist, ELEProbDist, sum_logs
from nltk.classify.api import ClassifierI

from django.utils.encoding import smart_str, smart_unicode

In [2]:
twlist = []

In [3]:
with open(r"C:\\Users\\Student21\\Documents\\Training\\Weather Project\\trained_weather_NoRT.csv", "r") as t:
    tweets_raw = pd.read_csv(t)

In [4]:
tweets = tweets_raw[['text', 'weather']].values.tolist()

In [5]:
twlist = [tuple(l) for l in tweets] # turn nested list of lists into list of tuples
twtokens = []

In [6]:
for (words, weather) in twlist:
#    words_filtered = [e.lower().decode('utf8') for e in words.split() if len(e) >= 3 and len(e) <= 10] # and <= 10
    words_filtered = [unicode(e.lower(), errors = 'replace') for e in words.split() if len(e) >= 3 and len(e) <= 10] # and <= 10
    twtokens.append((words_filtered, weather))

In [7]:
import nltk
def get_words_in_tweets(tweets):
    all_words = []
    for (words, weather) in tweets:
        all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

In [8]:
word_features = get_word_features(get_words_in_tweets(twtokens))

In [9]:
# The Classifier
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [10]:
training_set = nltk.classify.apply_features(extract_features, tweets)

In [11]:
# train the classifier
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [12]:
print classifier.show_most_informative_features(32)

Most Informative Features
    contains(summer���s) = False              no : yes    =      1.0 : 1.0
           contains(66%) = False              no : yes    =      1.0 : 1.0
      contains(forecast) = False              no : yes    =      1.0 : 1.0
     contains(@thedoors) = False              no : yes    =      1.0 : 1.0
      contains(@pc24fym) = False              no : yes    =      1.0 : 1.0
    contains(infinit...) = False              no : yes    =      1.0 : 1.0
       contains(finally) = False              no : yes    =      1.0 : 1.0
        contains(1012mb) = False              no : yes    =      1.0 : 1.0
       contains(provide) = False              no : yes    =      1.0 : 1.0
    contains(@moosawi17) = False              no : yes    =      1.0 : 1.0
      contains(mentally) = False              no : yes    =      1.0 : 1.0
          contains(#now) = False              no : yes    =      1.0 : 1.0
         contains(rainy) = False              no : yes    =      1.0 : 1.0

# Classification of London Tweets

In [32]:
with open(r"C:\\Users\\Student21\\Documents\\Training\\Weather Project\\Final Classify\\Clean_Tweets1.csv", "r") as t:
    data = pd.read_csv(t, encoding = 'utf8')

In [33]:
df = pd.DataFrame(data)

In [34]:
tx = df['text']#.str.encode('utf-8') #unicode(e.lower(), errors = 'replace')

In [35]:
tx #unicode(tx.lower(), errors = 'replace')

0       Great Smog of London: Mystery Of 1952 Fog That...
1       Great Smog of London: Mystery Of 1952 Fog That...
2       Great Smog of London: Mystery Of 1952 Fog That...
3       Great Smog of London: Mystery Of 1952 Fog That...
4       Great Smog of London: Mystery Of 1952 Fog That...
5       At last, scientists say they have solved the 1...
6       RT @tokenpet: Really Great Tips for Keeping Pe...
7       @LucyLondon @teamdb    Thank you for sharing @...
8       #ICE #CME #LME #LBMA - Finance Titans Face Off...
9       London listened to my weather wishes... WE HAV...
10      RT @hotdinners: Most instagrammable dish that ...
11      RT @hotdinners: Best dessert of 2016? We're st...
12      RT @SEENLONDONMAG: #Artist Bruce McLean is bac...
13      RT @frome_maude: Houses of Parliament in the \...
14      RT @TraderStef: #ICE #CME #LME #LBMA - Finance...
15      @philrichardsuk Fancy a bit of boiling blood P...
16      Those winter days when sun comes up and makes ...
17      "Myste

In [36]:
df['text'] = tx #unicode(tx, errors='ignore')

In [37]:
df.head()

,text,Date
0,Great Smog of London: Mystery Of 1952 Fog That...,2016-12-14
1,Great Smog of London: Mystery Of 1952 Fog That...,2016-12-14
2,Great Smog of London: Mystery Of 1952 Fog That...,2016-12-14
3,Great Smog of London: Mystery Of 1952 Fog That...,2016-12-14
4,Great Smog of London: Mystery Of 1952 Fog That...,2016-12-14


In [38]:
dffinal = df[['text']]
dffinal['date'] = df['Date']

In [39]:
dffinal.head()

,text,date
0,Great Smog of London: Mystery Of 1952 Fog That...,2016-12-14
1,Great Smog of London: Mystery Of 1952 Fog That...,2016-12-14
2,Great Smog of London: Mystery Of 1952 Fog That...,2016-12-14
3,Great Smog of London: Mystery Of 1952 Fog That...,2016-12-14
4,Great Smog of London: Mystery Of 1952 Fog That...,2016-12-14


In [47]:
for i in range(9453):
    if classifier.classify(extract_features((smart_str(df['text'][i])).split())) == 'yes':
        dffinal['text'][i] = smart_str(df['text'][i])
    else:
        dffinal['text'][i] = None

count = 0
for i in range(9453):
    if dffinal['text'][i] != None:
        count += 1

count2 = 0
dffinaltrained = pd.DataFrame({'date' : pd.Series(range(count), index=range(count)), 'text' : pd.Series(range(count), index=range(count))})
for i in range(9453):
    if dffinal['text'][i] != None:
        dffinaltrained['text'][count2] = dffinal['text'][i]
        dffinaltrained['date'][count2] = dffinal['date'][i]
        count2 += 1

C:\Users\Student21\Anaconda2\lib\site-packages\ipykernel\__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [48]:
dffinal

,text,date
0,None,2016-12-14
1,None,2016-12-14
2,None,2016-12-14
3,None,2016-12-14
4,None,2016-12-14
5,"At last, scientists say they have solved the 1...",2016-12-14
6,RT @tokenpet: Really Great Tips for Keeping Pe...,2016-12-14
7,@LucyLondon @teamdb Thank you for sharing @...,2016-12-14
8,None,2016-12-14
9,London listened to my weather wishes... WE HAV...,2016-12-14


In [49]:
dffinaltrained

,date,text
0,2016-12-14,"At last, scientists say they have solved the 1..."
1,2016-12-14,RT @tokenpet: Really Great Tips for Keeping Pe...
2,2016-12-14,@LucyLondon @teamdb Thank you for sharing @...
3,2016-12-14,London listened to my weather wishes... WE HAV...
4,2016-12-14,RT @hotdinners: Most instagrammable dish that ...
5,2016-12-14,RT @hotdinners: Best dessert of 2016? We're st...
6,2016-12-14,RT @SEENLONDONMAG: #Artist Bruce McLean is bac...
7,2016-12-14,RT @frome_maude: Houses of Parliament in the \...
8,2016-12-14,@philrichardsuk Fancy a bit of boiling blood P...
9,2016-12-14,Those winter days when sun comes up and makes ...


In [50]:
dffinaltrained.to_csv(path_or_buf="C:\\Users\\Student21\\Documents\\Training\\Weather Project\\Final Classify\\London_Classified_Tweets.csv", sep=',', na_rep='', \
               float_format=None, columns=['text', 'date'], header=True, index=False, index_label=None, mode='w', encoding=None, \
               compression=None, quoting=None, quotechar='"', line_terminator='\n', chunksize=None, tupleize_cols=False, \
               date_format=None, doublequote=True, escapechar=None, decimal='.')
# Change header = false and mode = a from header = true and mode = w